<a href="https://colab.research.google.com/github/yukismd/DL_for_ImageData_and_Finetuning/blob/main/text_classification_hatespeech/PT/japanese-roberta-base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
MODEL = 'rinna/japanese-roberta-base'

In [2]:
!pwd

/content


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls

drive  sample_data


In [5]:
!ls drive/MyDrive/experiment_results

hs_txt_classification


In [6]:
SAVE_DIR = 'drive/MyDrive/experiment_results/hs_txt_classification'

In [7]:
!pip install datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00


In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 7.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
#!pip install fugashi unidic_lite

In [ ]:
import datasets
print(datasets.__version__)
import transformers
print(transformers.__version__)
from transformers import AutoTokenizer
import evaluate
print(evaluate.__version__)

import torch
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import os
import json

## 学習データ, テストデータ

In [ ]:
df = pd.read_csv('https://sample-data-open.s3.ap-northeast-1.amazonaws.com/text_classification_hatespeech/hatespeech_train.csv')
print(df.shape)

In [ ]:
df_train = df[df['hold'] != 0]    # 学習データ
df_valid = df[df['hold'] == 0]    # 検証データ
df_train.shape, df_valid.shape

In [ ]:
ds_train = datasets.Dataset.from_pandas(df_train[['text','label']], preserve_index=False)
ds_valid = datasets.Dataset.from_pandas(df_valid[['text','label']], preserve_index=False)

In [ ]:
df_test = pd.read_csv('https://sample-data-open.s3.ap-northeast-1.amazonaws.com/text_classification_hatespeech/hatespeech_test.csv')
print(df_test.shape)

(600, 4)


In [ ]:
ds_test = datasets.Dataset.from_pandas(df_test[['text','label']], preserve_index=False)

## トークナイザー

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)
tokenizer

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/251 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/231k [00:00<?, ?B/s]

BertJapaneseTokenizer(name_or_path='tohoku-nlp/bert-large-japanese-v2', vocab_size=32768, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"],
                     truncation=True,
                     max_length=128)    # 学習データの文長が短いので128に設定

In [ ]:
ds_tkn_train = ds_train.map(tokenize_function, batched=True)
ds_tkn_valid = ds_valid.map(tokenize_function, batched=True)
ds_tkn_train, ds_tkn_valid

Parameter 'function'=<function tokenize_function at 0x7feb543d0310> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/3759 [00:00<?, ? examples/s]

Map:   0%|          | 0/897 [00:00<?, ? examples/s]

(Dataset({
     features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 3759
 }),
 Dataset({
     features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 897
 }))

In [ ]:
# textカラムの削除
ds_tkn_train = ds_tkn_train.remove_columns(['text'])
ds_tkn_valid = ds_tkn_valid.remove_columns(['text'])

# labelカラムをlabelsへリネーム
ds_tkn_train = ds_tkn_train.rename_column('label', 'labels')
ds_tkn_valid = ds_tkn_valid.rename_column('label', 'labels')

In [ ]:
ds_tkn_test = ds_test.map(tokenize_function, batched=True)
ds_tkn_test

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 600
})

In [ ]:
# textカラムの削除
ds_tkn_test = ds_tkn_test.remove_columns(['text'])

# labelカラムをlabelsへリネーム
ds_tkn_test = ds_tkn_test.rename_column('label', 'labels')

## 学習

In [ ]:
metric = evaluate.load("f1")

# 精度評価関数
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
def do_training():
  print('N_EPOCH: ', N_EPOCH)
  print('BATCH_SIZE: ', BATCH_SIZE)
  print('LR', LR)

  model = AutoModelForSequenceClassification.from_pretrained(MODEL,
                                                           num_labels=2)

  trainer_args = TrainingArguments(
    seed=np.random.randint(0, 9999),
    output_dir=SAVE_DIR,
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    #save_strategy="epoch",
    save_steps=1e6, # saveのステップを大きくしてここではモデルの保存を行わないようにする
    log_level="critical",
    num_train_epochs=N_EPOCH,
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=12,
    save_total_limit=1,
    fp16=True,
    remove_unused_columns=False,
    report_to="none"
    )

  trainer = Trainer(
    model=model,
    args=trainer_args,
    tokenizer=tokenizer,
    train_dataset=ds_tkn_train,
    eval_dataset=ds_tkn_valid,
    compute_metrics=compute_metrics,
    )

  #return trainer

  print('----------  Start Training  ----------')
  trainer.train()

  print('----------  Res on Valid  ----------')
  res_json = trainer.evaluate()
  print(res_json)

  print('----------  Res on Test  ----------')
  test_preds = trainer.predict(ds_tkn_test)
  test_metric = test_preds.metrics
  print(test_metric)

  return res_json, test_metric


In [ ]:
N_EPOCH = 3
BATCH_SIZE = 16
LR = 0.000003  # 3e-6

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

Experiment Name:  bert-large-japanese-v2_Epoch3_Batch16_Lr3e-06
N_EPOCH:  3
BATCH_SIZE:  16
LR 3e-06


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at tohoku-nlp/bert-large-japanese-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------


Epoch,Training Loss,Validation Loss,F1
1,0.200900,0.120539,0.113208
2,0.107100,0.102781,0.620690
3,0.072400,0.105454,0.659341


----------  Res on Valid  ----------


{'eval_loss': 0.1054539829492569, 'eval_f1': 0.6593406593406594, 'eval_runtime': 3.2453, 'eval_samples_per_second': 276.397, 'eval_steps_per_second': 23.11, 'epoch': 3.0}
----------  Res on Test  ----------
{'test_loss': 0.12902244925498962, 'test_f1': 0.591549295774648, 'test_runtime': 2.174, 'test_samples_per_second': 275.988, 'test_steps_per_second': 22.999}


In [ ]:
# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print(save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print(save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

drive/MyDrive/experiment_results/hs_txt_classification/VALID__bert-large-japanese-v2_Epoch3_Batch16_Lr3e-06.json
drive/MyDrive/experiment_results/hs_txt_classification/TEST__bert-large-japanese-v2_Epoch3_Batch16_Lr3e-06.json


In [ ]:
!ls drive/MyDrive/experiment_results/hs_txt_classification

checkpoint-940	TEST__bert-large-japanese-v2_Epoch3_Batch16_Lr3e-06.json
OLD		VALID__bert-large-japanese-v2_Epoch3_Batch16_Lr3e-06.json
OLD2


In [ ]:
N_EPOCH = 3
BATCH_SIZE = 16
LR = 0.00001  # 1e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print('Val Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print('Test Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  bert-large-japanese-v2_Epoch3_Batch16_Lr1e-05
N_EPOCH:  3
BATCH_SIZE:  16
LR 1e-05


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------
{'loss': 0.1709, 'grad_norm': 2.8458921909332275, 'learning_rate': 6.695035460992908e-06, 'epoch': 1.0}
{'eval_loss': 0.11040271073579788, 'eval_f1': 0.6153846153846154, 'eval_runtime': 3.4937, 'eval_samples_per_second': 256.747, 'eval_steps_per_second': 21.467, 'epoch': 1.0}
{'loss': 0.0658, 'grad_norm': 0.06786464899778366, 'learning_rate': 3.3617021276595745e-06, 'epoch': 2.0}
{'eval_loss': 0.14480413496494293, 'eval_f1': 0.6813186813186813, 'eval_runtime': 3.1322, 'eval_samples_per_second': 286.381, 'eval_steps_per_second': 23.945, 'epoch': 2.0}
{'loss': 0.0146, 'grad_norm': 0.03168032690882683, 'learning_rate': 2.836879432624114e-08, 'epoch': 3.0}
{'eval_loss': 0.19470055401325226, 'eval_f1': 0.6238532110091743, 'eval_runtime': 3.2396, 'eval_samples_per_second': 276.884, 'eval_steps_per_second': 23.151, 'epoch': 3.0}
{'train_runtime': 237.1888, 'train_samples_per_second': 47.544, 'train_steps_per_second': 2.972, 'train_loss': 0.08376785136283

In [ ]:
N_EPOCH = 3
BATCH_SIZE = 16
LR = 0.00003  # 3e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print('Val Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print('Test Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  bert-large-japanese-v2_Epoch3_Batch16_Lr3e-05
N_EPOCH:  3
BATCH_SIZE:  16
LR 3e-05


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------
{'loss': 0.1841, 'grad_norm': 1.7147047519683838, 'learning_rate': 2.0085106382978722e-05, 'epoch': 1.0}
{'eval_loss': 0.1149047464132309, 'eval_f1': 0.5918367346938774, 'eval_runtime': 3.126, 'eval_samples_per_second': 286.95, 'eval_steps_per_second': 23.992, 'epoch': 1.0}
{'loss': 0.0727, 'grad_norm': 0.46630656719207764, 'learning_rate': 1.0085106382978723e-05, 'epoch': 2.0}
{'eval_loss': 0.16124427318572998, 'eval_f1': 0.6296296296296295, 'eval_runtime': 3.3514, 'eval_samples_per_second': 267.649, 'eval_steps_per_second': 22.379, 'epoch': 2.0}
{'loss': 0.0093, 'grad_norm': 0.010333187878131866, 'learning_rate': 8.510638297872341e-08, 'epoch': 3.0}
{'eval_loss': 0.20593871176242828, 'eval_f1': 0.6153846153846154, 'eval_runtime': 3.1474, 'eval_samples_per_second': 284.996, 'eval_steps_per_second': 23.829, 'epoch': 3.0}
{'train_runtime': 230.5219, 'train_samples_per_second': 48.919, 'train_steps_per_second': 3.058, 'train_loss': 0.088723792082874

In [ ]:
N_EPOCH = 3
BATCH_SIZE = 16
LR = 0.00005  # 5e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print('Val Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print('Test Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  bert-large-japanese-v2_Epoch3_Batch16_Lr5e-05
N_EPOCH:  3
BATCH_SIZE:  16
LR 5e-05


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------
{'loss': 0.1849, 'grad_norm': 0.1489768922328949, 'learning_rate': 3.347517730496454e-05, 'epoch': 1.0}
{'eval_loss': 0.2075231671333313, 'eval_f1': 0.0392156862745098, 'eval_runtime': 3.1383, 'eval_samples_per_second': 285.826, 'eval_steps_per_second': 23.899, 'epoch': 1.0}
{'loss': 0.1001, 'grad_norm': 0.22944918274879456, 'learning_rate': 1.6808510638297873e-05, 'epoch': 2.0}
{'eval_loss': 0.17644433677196503, 'eval_f1': 0.588235294117647, 'eval_runtime': 3.2315, 'eval_samples_per_second': 277.579, 'eval_steps_per_second': 23.209, 'epoch': 2.0}
{'loss': 0.0235, 'grad_norm': 0.016286130994558334, 'learning_rate': 1.4184397163120568e-07, 'epoch': 3.0}
{'eval_loss': 0.21076659858226776, 'eval_f1': 0.5952380952380952, 'eval_runtime': 3.1296, 'eval_samples_per_second': 286.616, 'eval_steps_per_second': 23.965, 'epoch': 3.0}
{'train_runtime': 233.6047, 'train_samples_per_second': 48.274, 'train_steps_per_second': 3.018, 'train_loss': 0.10280947516150

#### Batch Size -> 32

In [ ]:
N_EPOCH = 3
BATCH_SIZE = 32
LR = 0.000003  # 3e-6

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print(save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print(save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  bert-large-japanese-v2_Epoch3_Batch32_Lr3e-06
N_EPOCH:  3
BATCH_SIZE:  32
LR 3e-06


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------
{'loss': 0.2206, 'grad_norm': 2.7110583782196045, 'learning_rate': 2e-06, 'epoch': 1.0}
{'eval_loss': 0.1318766474723816, 'eval_f1': 0.24137931034482757, 'eval_runtime': 3.3516, 'eval_samples_per_second': 267.635, 'eval_steps_per_second': 22.378, 'epoch': 1.0}
{'loss': 0.1229, 'grad_norm': 6.040450096130371, 'learning_rate': 1e-06, 'epoch': 2.0}
{'eval_loss': 0.115012988448143, 'eval_f1': 0.5, 'eval_runtime': 3.2829, 'eval_samples_per_second': 273.236, 'eval_steps_per_second': 22.846, 'epoch': 2.0}
{'loss': 0.0906, 'grad_norm': 14.454383850097656, 'learning_rate': 8.474576271186441e-09, 'epoch': 3.0}
{'eval_loss': 0.09934377670288086, 'eval_f1': 0.6352941176470589, 'eval_runtime': 3.2553, 'eval_samples_per_second': 275.554, 'eval_steps_per_second': 23.04, 'epoch': 3.0}
{'train_runtime': 163.342, 'train_samples_per_second': 69.039, 'train_steps_per_second': 2.167, 'train_loss': 0.1446775932096492, 'epoch': 3.0}
----------  Res on Valid  ----------


In [ ]:
N_EPOCH = 3
BATCH_SIZE = 32
LR = 0.00001  # 1e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print('Val Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print('Test Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  bert-large-japanese-v2_Epoch3_Batch32_Lr1e-05
N_EPOCH:  3
BATCH_SIZE:  32
LR 1e-05


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------
{'loss': 0.1833, 'grad_norm': 3.771418809890747, 'learning_rate': 6.694915254237288e-06, 'epoch': 1.0}
{'eval_loss': 0.10110864043235779, 'eval_f1': 0.6041666666666666, 'eval_runtime': 3.1506, 'eval_samples_per_second': 284.707, 'eval_steps_per_second': 23.805, 'epoch': 1.0}
{'loss': 0.074, 'grad_norm': 2.681330680847168, 'learning_rate': 3.361581920903955e-06, 'epoch': 2.0}
{'eval_loss': 0.176632821559906, 'eval_f1': 0.5496183206106869, 'eval_runtime': 3.1402, 'eval_samples_per_second': 285.653, 'eval_steps_per_second': 23.884, 'epoch': 2.0}
{'loss': 0.0261, 'grad_norm': 0.18965402245521545, 'learning_rate': 2.824858757062147e-08, 'epoch': 3.0}
{'eval_loss': 0.17666734755039215, 'eval_f1': 0.607843137254902, 'eval_runtime': 3.2164, 'eval_samples_per_second': 278.885, 'eval_steps_per_second': 23.318, 'epoch': 3.0}
{'train_runtime': 163.254, 'train_samples_per_second': 69.076, 'train_steps_per_second': 2.168, 'train_loss': 0.09445475387034444, 'epo

In [ ]:
N_EPOCH = 3
BATCH_SIZE = 32
LR = 0.00003  # 3e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print('Val Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print('Test Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  bert-large-japanese-v2_Epoch3_Batch32_Lr3e-05
N_EPOCH:  3
BATCH_SIZE:  32
LR 3e-05


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------
{'loss': 0.1623, 'grad_norm': 0.06856409460306168, 'learning_rate': 2.0084745762711865e-05, 'epoch': 1.0}
{'eval_loss': 0.10587729513645172, 'eval_f1': 0.5714285714285714, 'eval_runtime': 3.1714, 'eval_samples_per_second': 282.841, 'eval_steps_per_second': 23.649, 'epoch': 1.0}
{'loss': 0.0718, 'grad_norm': 5.815360069274902, 'learning_rate': 1.0084745762711864e-05, 'epoch': 2.0}
{'eval_loss': 0.12482735514640808, 'eval_f1': 0.5977011494252874, 'eval_runtime': 3.1206, 'eval_samples_per_second': 287.448, 'eval_steps_per_second': 24.034, 'epoch': 2.0}
{'loss': 0.0102, 'grad_norm': 0.03503182902932167, 'learning_rate': 8.47457627118644e-08, 'epoch': 3.0}
{'eval_loss': 0.1689557284116745, 'eval_f1': 0.6521739130434783, 'eval_runtime': 3.1228, 'eval_samples_per_second': 287.244, 'eval_steps_per_second': 24.017, 'epoch': 3.0}
{'train_runtime': 162.0109, 'train_samples_per_second': 69.606, 'train_steps_per_second': 2.185, 'train_loss': 0.0814184226558706

In [ ]:
N_EPOCH = 3
BATCH_SIZE = 32
LR = 0.00005  # 5e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print('Val Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print('Test Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  bert-large-japanese-v2_Epoch3_Batch32_Lr5e-05
N_EPOCH:  3
BATCH_SIZE:  32
LR 5e-05


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------
{'loss': 0.1539, 'grad_norm': 10.958085060119629, 'learning_rate': 3.3333333333333335e-05, 'epoch': 1.0}
{'eval_loss': 0.11710278689861298, 'eval_f1': 0.5066666666666666, 'eval_runtime': 3.2054, 'eval_samples_per_second': 279.842, 'eval_steps_per_second': 23.398, 'epoch': 1.0}
{'loss': 0.0429, 'grad_norm': 0.17741163074970245, 'learning_rate': 1.6666666666666667e-05, 'epoch': 2.0}
{'eval_loss': 0.15513212978839874, 'eval_f1': 0.6341463414634146, 'eval_runtime': 3.1618, 'eval_samples_per_second': 283.698, 'eval_steps_per_second': 23.721, 'epoch': 2.0}
{'loss': 0.005, 'grad_norm': 0.004832438658922911, 'learning_rate': 0.0, 'epoch': 3.0}
{'eval_loss': 0.18287663161754608, 'eval_f1': 0.6190476190476191, 'eval_runtime': 3.32, 'eval_samples_per_second': 270.178, 'eval_steps_per_second': 22.59, 'epoch': 3.0}
{'train_runtime': 163.2426, 'train_samples_per_second': 69.081, 'train_steps_per_second': 2.169, 'train_loss': 0.06726480052296051, 'epoch': 3.0}
-

#### Top3の繰り返し

In [ ]:
N_EPOCH = 3
BATCH_SIZE = 32
LR = 0.00001  # 1e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print('Val Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print('Test Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  bert-large-japanese-v2_Epoch3_Batch32_Lr1e-05
N_EPOCH:  3
BATCH_SIZE:  32
LR 1e-05


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------
{'loss': 0.1811, 'grad_norm': 4.656700134277344, 'learning_rate': 6.666666666666667e-06, 'epoch': 1.0}
{'eval_loss': 0.09850490838289261, 'eval_f1': 0.6857142857142857, 'eval_runtime': 4.3911, 'eval_samples_per_second': 204.278, 'eval_steps_per_second': 17.08, 'epoch': 1.0}
{'loss': 0.071, 'grad_norm': 5.024030685424805, 'learning_rate': 3.3333333333333333e-06, 'epoch': 2.0}
{'eval_loss': 0.11538079380989075, 'eval_f1': 0.6407766990291262, 'eval_runtime': 3.2738, 'eval_samples_per_second': 273.995, 'eval_steps_per_second': 22.909, 'epoch': 2.0}
{'loss': 0.0238, 'grad_norm': 2.2081689834594727, 'learning_rate': 0.0, 'epoch': 3.0}
{'eval_loss': 0.13456352055072784, 'eval_f1': 0.7083333333333334, 'eval_runtime': 3.2883, 'eval_samples_per_second': 272.786, 'eval_steps_per_second': 22.808, 'epoch': 3.0}
{'train_runtime': 166.7613, 'train_samples_per_second': 67.624, 'train_steps_per_second': 2.123, 'train_loss': 0.09197215980055642, 'epoch': 3.0}
-----

In [ ]:
N_EPOCH = 3
BATCH_SIZE = 32
LR = 0.00003  # 3e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print('Val Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print('Test Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  bert-large-japanese-v2_Epoch3_Batch32_Lr3e-05
N_EPOCH:  3
BATCH_SIZE:  32
LR 3e-05


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------
{'loss': 0.1655, 'grad_norm': 4.404618740081787, 'learning_rate': 2.016949152542373e-05, 'epoch': 1.0}
{'eval_loss': 0.12443897873163223, 'eval_f1': 0.5894736842105264, 'eval_runtime': 3.1313, 'eval_samples_per_second': 286.46, 'eval_steps_per_second': 23.951, 'epoch': 1.0}
{'loss': 0.0637, 'grad_norm': 0.5565434098243713, 'learning_rate': 1.016949152542373e-05, 'epoch': 2.0}
{'eval_loss': 0.13931584358215332, 'eval_f1': 0.6101694915254237, 'eval_runtime': 3.1267, 'eval_samples_per_second': 286.887, 'eval_steps_per_second': 23.987, 'epoch': 2.0}
{'loss': 0.0164, 'grad_norm': 0.04297836497426033, 'learning_rate': 1.694915254237288e-07, 'epoch': 3.0}
{'eval_loss': 0.16421248018741608, 'eval_f1': 0.6292134831460674, 'eval_runtime': 3.2123, 'eval_samples_per_second': 279.238, 'eval_steps_per_second': 23.348, 'epoch': 3.0}
{'train_runtime': 162.5879, 'train_samples_per_second': 69.359, 'train_steps_per_second': 2.177, 'train_loss': 0.08184480027290388,

In [ ]:
N_EPOCH = 3
BATCH_SIZE = 32
LR = 0.00005  # 5e-5

experiment_name = '{}_Epoch{}_Batch{}_Lr{}'.format(MODEL.split('/')[1], N_EPOCH, BATCH_SIZE, LR)
print('Experiment Name: ', experiment_name)

res_val, res_test = do_training()

# Save Valid Result
save_file = os.path.join(SAVE_DIR, 'VALID__' + experiment_name + '.json')
print('Val Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_val, f, indent=2)

# Save Test Result
save_file = os.path.join(SAVE_DIR, 'TEST__' + experiment_name + '.json')
print('Test Res Save File: ', save_file)
with open(save_file, 'w') as f:
    json.dump(res_test, f, indent=2)

Experiment Name:  bert-large-japanese-v2_Epoch3_Batch32_Lr5e-05
N_EPOCH:  3
BATCH_SIZE:  32
LR 5e-05


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


----------  Start Training  ----------
{'loss': 0.185, 'grad_norm': 16.469192504882812, 'learning_rate': 3.3333333333333335e-05, 'epoch': 1.0}
{'eval_loss': 0.1604452282190323, 'eval_f1': 0.21428571428571425, 'eval_runtime': 3.1392, 'eval_samples_per_second': 285.739, 'eval_steps_per_second': 23.891, 'epoch': 1.0}
{'loss': 0.0724, 'grad_norm': 5.132132053375244, 'learning_rate': 1.6666666666666667e-05, 'epoch': 2.0}
{'eval_loss': 0.19136753678321838, 'eval_f1': 0.6206896551724138, 'eval_runtime': 4.1566, 'eval_samples_per_second': 215.803, 'eval_steps_per_second': 18.044, 'epoch': 2.0}
{'loss': 0.0109, 'grad_norm': 0.00902633834630251, 'learning_rate': 0.0, 'epoch': 3.0}
{'eval_loss': 0.23452244699001312, 'eval_f1': 0.6407766990291262, 'eval_runtime': 3.1278, 'eval_samples_per_second': 286.781, 'eval_steps_per_second': 23.978, 'epoch': 3.0}
{'train_runtime': 164.9919, 'train_samples_per_second': 68.349, 'train_steps_per_second': 2.146, 'train_loss': 0.08943819662945418, 'epoch': 3.0}
-